# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [38]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
             "keyword" : "hotel",
             "radius" : 5000,
             "key" : g_key }

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_df = pd.read_csv(r"C:\Users\prath\Desktop\python-api\python-api-challenge\output_data\cities.csv")
weather_df.dropna(inplace = True) 

# Visualise
print(len(weather_df))
weather_df




548


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,kapaa,22.0752,-159.3190,26.99,77,40,6.17,US,1651608887
1,1,barrow,71.2906,-156.7887,-5.99,86,100,5.14,US,1651608707
2,2,busselton,-33.6500,115.3333,13.33,68,84,4.85,AU,1651608889
3,3,mar del plata,-38.0023,-57.5575,16.08,59,0,6.17,AR,1651608890
4,4,vaini,-21.2000,-175.2000,23.09,88,20,2.57,TO,1651608890
...,...,...,...,...,...,...,...,...,...,...
546,546,poshekhonye,58.4993,39.1353,2.33,90,100,3.29,RU,1651609201
547,547,santa rosalia,27.3167,-112.2833,26.61,33,98,2.33,MX,1651609201
548,548,boqueirao,-7.5000,-36.1333,26.75,73,25,5.61,BR,1651609202
549,549,panyutyne,48.9333,36.2833,10.82,56,29,2.27,UA,1651609203


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)

In [4]:
#Use the Lat and Lng as locations and Humidity as the weight.

# Store latitude and longitude in locations
locations = weather_df[["Lat", "Lng"]]

#Humidity as weight
humidity = weather_df["Humidity"].astype(float)


In [5]:
#Add Heatmap layer to map

fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
narrow_city_df = weather_df.loc[(weather_df["Wind Speed"] <=4.5) & (weather_df["Cloudiness"] == 0) & \
                               (weather_df["Max Temp"] >=21) & (weather_df["Max Temp"]<= 27)].dropna()



#visualize
narrow_city_df

#perfect_weather_df = perfect_weather_df.loc[perfect_weather_df["Max Temp"] < 27]
#perfect_weather_df = perfect_weather_df.loc[perfect_weather_df["Max Temp"] > 21]
#perfect_weather_df = perfect_weather_df.loc[perfect_weather_df["Wind Speed"] < 4.5]
#perfect_weather_df = perfect_weather_df.loc[perfect_weather_df["Cloudiness"] == 0]


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
32,32,prado,-17.3411,-39.2208,25.18,73,0,3.11,BR,1651608908
68,68,sao filipe,14.8961,-24.4956,22.95,76,0,4.32,CV,1651608933
126,126,kingman,35.1894,-114.0530,23.96,8,0,4.12,US,1651608887
135,135,caravelas,-17.7125,-39.2481,25.08,72,0,2.54,BR,1651608969
157,157,zhuhai,22.2769,113.5678,22.84,68,0,3.09,CN,1651608986
210,210,catamarca,-28.4696,-65.7852,24.32,31,0,0.89,AR,1651609019
233,233,santiago del estero,-27.7951,-64.2615,21.52,31,0,1.49,AR,1651608787
278,278,madimba,-10.4000,40.3333,23.61,86,0,4.17,TZ,1651609057
346,346,tucuman,-26.8241,-65.2226,22.00,40,0,2.06,AR,1651609092
353,353,karratha,-20.7377,116.8463,26.23,58,0,1.66,AU,1651609095


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [27]:
#Store into variable named hotel_df
hotel_df = narrow_city_df.loc[:,["City", "Country", "Lat", "Lng"]]

#Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotal Name"] = ""

#result

hotel_df

,City,Country,Lat,Lng,Hotal Name
32,prado,BR,-17.3411,-39.2208,
68,sao filipe,CV,14.8961,-24.4956,
126,kingman,US,35.1894,-114.0530,
135,caravelas,BR,-17.7125,-39.2481,
157,zhuhai,CN,22.2769,113.5678,
210,catamarca,AR,-28.4696,-65.7852,
233,santiago del estero,AR,-27.7951,-64.2615,
278,madimba,TZ,-10.4000,40.3333,
346,tucuman,AR,-26.8241,-65.2226,
353,karratha,AU,-20.7377,116.8463,


In [45]:
for index, row in hotel_df.iterrows():
    #get city name
    #get Lat &Lnt from DataFrame
    
    lat = row ["Lat"]
    lng = row ["Lng"]
    city_name = row["City"]
    
    #add Params dict
    
    params["location"] = f"{lat},{lng}"
    
    #print rerieving data
    
    print(f"Retrieving result for index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
   
    print("------------")
    # Print end of results
    print("End of results")
    print("--------------")
    

Retrieving result for index 32: prado.
Closest hotel in prado is POUSADA PONTA DE AREIA - PRADO - BAHIA - HOSPEDAGEM E HOTEL.
------------
End of results
--------------
Retrieving result for index 68: sao filipe.
Closest hotel in sao filipe is Hotel Savana.
------------
End of results
--------------
Retrieving result for index 126: kingman.
Closest hotel in kingman is Home2 Suites by Hilton Kingman.
------------
End of results
--------------
Retrieving result for index 135: caravelas.
Closest hotel in caravelas is Pousada Liberdade.
------------
End of results
--------------
Retrieving result for index 157: zhuhai.
Closest hotel in zhuhai is 萬豪酒店.
------------
End of results
--------------
Retrieving result for index 210: catamarca.
Closest hotel in catamarca is Amerian Catamarca Park Hotel.
------------
End of results
--------------
Retrieving result for index 233: santiago del estero.
Closest hotel in santiago del estero is Hilton Garden Inn Santiago del Estero.
------------
End of r

In [40]:
hotel_df

,City,Country,Lat,Lng,Hotal Name,Hotel Name
32,prado,BR,-17.3411,-39.2208,,POUSADA PONTA DE AREIA - PRADO - BAHIA - HOSPE...
68,sao filipe,CV,14.8961,-24.4956,,Hotel Xaguate
126,kingman,US,35.1894,-114.0530,,Home2 Suites by Hilton Kingman
135,caravelas,BR,-17.7125,-39.2481,,Pousada Liberdade
157,zhuhai,CN,22.2769,113.5678,,Dehan International Hotel
210,catamarca,AR,-28.4696,-65.7852,,Amerian Catamarca Park Hotel
233,santiago del estero,AR,-27.7951,-64.2615,,Hotel Libertador
278,madimba,TZ,-10.4000,40.3333,,NaN
346,tucuman,AR,-26.8241,-65.2226,,Sheraton Tucuman Hotel
353,karratha,AU,-20.7377,116.8463,,Karratha International Hotel


In [41]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [44]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))